In [6]:
# Import
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
from gensim.models import KeyedVectors
import pickle

In [7]:
train_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/smith/original/train/eng_ita.tsv'

eng_emb = '/mnt/permanent/Language/Multi/FB/wiki.en/wiki.en.vec'
ita_emb = '/mnt/permanent/Language/Multi/FB/wiki.it/wiki.it.vec'

limit = None

tr_en_fn = 'train_eng.pickle'

tr_it_fn = 'train_ita.pickle'

In [8]:
def read_emb(emb_fn, limit):
    model = KeyedVectors.load_word2vec_format(emb_fn, binary=False, limit=limit)
    model.syn0 /= np.sqrt((model.syn0 ** 2).sum(1))[:, None]
    return model

In [9]:
def read_word_pairs_tsv(fn, id1, id2):
    wl1 = set()
    wl2 = set()
    with open(fn) as f:
        lines = f.readlines()
        data = [(line.split()[id1], line.split()[id2]) for i, line in enumerate(lines)]
    for (w1, w2) in data:
        wl1.add(w1)
        wl2.add(w2)
    return data, wl1, wl2

In [10]:
# saving format: (vocab: ordered list according to frequency, emb_dict: word - nparray dictionary)
def get_filtered_embedding(emb, wl, fn):
    to_save = []
    for w in wl:
        if w in emb:
            to_save.append([w, emb[w], emb.index2word.index(w)])
        else:
            print('not found: {}'.format(w))
    to_save.sort(key=lambda x: x[2])
    dim = 300
    vocab = [l[0] for l in to_save]
    filtered_mod = dict()
    for i, w in enumerate(vocab):
        filtered_mod[w] = emb[w]
    with open(fn, 'wb') as f:
        pickle.dump(file=f, obj=filtered_mod)
    return filtered_mod

In [11]:
m_en = read_emb(eng_emb, limit)
m_it = read_emb(ita_emb, limit)

In [12]:
train_wp, tr_en, tr_it = read_word_pairs_tsv(train_fn, 0, 1)

In [13]:
# En
print('en train')
m_en_tr_fil = get_filtered_embedding(m_en, tr_en, tr_en_fn)

# It
print('it train')
m_it_tr_fil = get_filtered_embedding(m_it, tr_it, tr_it_fn)

en train
it train
not found: prelaurea


In [14]:
def load(fn):
    with open(fn, 'rb') as f:
        data = pickle.load(f)
    return data

In [15]:
def test_load(fn, message):
    print(message)
    emb = load(fn)
    print('vocab length: {}'.format(len(emb.keys())))

In [16]:
test_load(tr_en_fn, 'train en')
test_load(tr_it_fn, 'train it')

train en
vocab length: 3442
train it
vocab length: 4548
